# Rachel Ward - DAV 5400

## Week 8 Assignment

In this week's assignment I will be using regular expressions "regex" to gather and manipulate data.

### Question 1:

Given the below string, extract the names of each individual and store them in a vector:

"555-1239Moe Szyslak(636) 555-0113Burns, C. Montgomery555 -6542Rev. Timothy Lovejoy555
8904Ned Flanders636-555-3226Simpson, Homer5553642Dr. Julius Hibbert"

To accomplish this I will use the regex findall() function to return all the strings that do not have parentheses, numbers, or dashes. This will return some strings which are just spaces, so I will then filter those out using the strip function in a list comprehension.

In [224]:
#first import the regex module using import re
import re

#also import pandas
import pandas as pd

#also import numpy
import numpy as np

#Next I assign the string to a variable so that I can reference it later on
mess = "555-1239Moe Szyslak(636) 555-0113Burns, C. Montgomery555 -6542Rev. Timothy Lovejoy555 8904Ned Flanders636-555-3226Simpson, Homer5553642Dr. Julius Hibbert"

#I will use the regex function findall to find all strings that do not have parentheses, numbers, or dashes.
#I will assign this to a variable
#Note this will generate some extra items because it will pick up spaces after parentheses. I will remove those later
listofnames = re.findall(r"[^()0-9-]+", mess)

#use list comprehension and the strip function to get rid of the extra items which contain only spaces
cleannames = [i.strip() for i in listofnames if i.strip()]

#show the list
cleannames

['Moe Szyslak',
 'Burns, C. Montgomery',
 'Rev. Timothy Lovejoy',
 'Ned Flanders',
 'Simpson, Homer',
 'Dr. Julius Hibbert']

### Question 2:

Using your new vector containing only the names of the six individuals, complete the following tasks:
* a. Use your regex skills to rearrange the vector so that all elements conform to the standard “firstname
lastname”, preserving any titles (e.g., “Rev.”, “Dr.”, etc) or middle/second names.
* b. Construct a logical vector indicating whether a character has a title (i.e., Rev. and Dr.).
* c. Construct a logical vector indicating whether a character has a middle/second name.

### Question 2a:
First I will construct a new list which is empty. Then I will create a regex expression that identifies the string up to and including the comma. Then I will use a for loop to iterate through the items in the list created above, check if there is a match to my regex, and if so, split the string and remove the spaces, then create a new item and join the pieces in the correct order and add it to the empty list. Otherwise, if the item does not contain a match, the for loop will just add the item (without adjustment) to the new list. Then I will show the new list of the names in the correct order.

In [187]:
#create an empty list to which items will be appended using the for loop below
newlist = []

#create a regex to get any text up to the first comma and assign it to a variable for ease of use
commaexp = '^(.+?),'

#create a for loop which cycles through the items in the cleannames list from above
for i in cleannames:
    #look to see if the list item has a match to my regex
    if re.match(commaexp,i):
        #if there is a match, split the item into sub strings by the comma and strip the extra spaces
        changed = [x.strip() for x in i.split(',')]
        #create a new item that concatenates the item from changed at index 1, a space, and the item at index 0
        newitem = changed[1]+ ' ' + changed[0]
        #append the new item to the list
        newlist.append(newitem)
    else:
        #if there is not a match, just add the item as it was, no change necessary
        newlist.append(i)

#show the new list
newlist


['Moe Szyslak',
 'C. Montgomery Burns',
 'Rev. Timothy Lovejoy',
 'Ned Flanders',
 'Homer Simpson',
 'Dr. Julius Hibbert']

### Question 2b:

"Construct a logical vector indicating whether a character has a title (i.e. Rev and Dr.)."

To accomplish this I will first create a Pandas Series using the new list I created above with the names in the correct order. Then I will use the built in Pandas string function contains() on my Series to search for the strings Dr. or Rev. This will generate a boolean vector. I will make this it's own Series, and then add it to the original Series so the results will be easier to interpret.

In [225]:
#make the list "newlist" created in part 2a into a series in pandas
namesframe = pd.Series(newlist)

#use the str.contains() function and an or to create a boolean vector which tells whether or not each character has a title
#note: searches only for Dr. or Rev.
addcolumn = namesframe.str.contains('Dr.|Rev.')

#create new Pandas Series with the boolean results
addcolumn=pd.Series(addcolumn)

#make a list of the series to concatenate
newframe = [namesframe,addcolumn]

#concatenate the two series into a data frame with the names in one column and the boolean results in another column
result = pd.concat(newframe, axis =1)

#rename columns to make results legible
result = result.rename(columns={0: "Characters", 1: "Has Title?"})

#show the results
result

,Characters,Has Title?
0,Moe Szyslak,False
1,C. Montgomery Burns,False
2,Rev. Timothy Lovejoy,True
3,Ned Flanders,False
4,Homer Simpson,False
5,Dr. Julius Hibbert,True


### Question 2c:
"Construct a logical vector indicating whether a character has a middle/second name."

For this task I will first create a regex which will determine whether a character has a title. If the character has a title, I will replace the title with a "".
Then I will create another regex which looks for patterns that are a string of letters. I will use this regex to count the number of pattern matches this has, to determine if someone has three names. 
I will make this into a dataframe so the results are more easily understood.

Lastly, I will add a boolean column to this new data frame which says "True" only when there are 3 pattern matches for the character.

In [212]:
#create a regex string to detect the two titles
title = r"Rev.|Dr."

#create a new series where the titles are replaced with "" 
removedtitles = namesframe.str.replace(title, repl="")

#show this new series
removedtitles

0            Moe Szyslak
1    C. Montgomery Burns
2        Timothy Lovejoy
3           Ned Flanders
4          Homer Simpson
5         Julius Hibbert
dtype: object

In [221]:
#create a regex for strings of letters
letters = r"[A-Za-z]+"

#count the number of occurrences of the letters pattern as described above
#any item with three pattern matches will have a middle name
threenames = removedtitles.str.count(letters)

#create new Pandas Series with the boolean results
threenames=pd.Series(threenames)

#make a list of the series to concatenate
cframe = [removedtitles,threenames]

#concatenate the two series into a data frame with the names with titles removed in one column
#and the boolean results in another column
cresult = pd.concat(cframe, axis =1)

#rename columns to make results legible
cresult = cresult.rename(columns={0: "Characters w/o Titles", 1: "# of pattern matches"})

#show the results
cresult


,Characters w/o Titles,# of pattern matches
0,Moe Szyslak,2
1,C. Montgomery Burns,3
2,Timothy Lovejoy,2
3,Ned Flanders,2
4,Homer Simpson,2
5,Julius Hibbert,2


In [226]:
#create a new column which returns True if the number of pattern matches is 3
cresult['Has Three Names?'] = cresult['# of pattern matches'] ==3

#show the results
cresult

,Characters w/o Titles,# of pattern matches,Has Three Names?
0,Moe Szyslak,2,False
1,C. Montgomery Burns,3,True
2,Timothy Lovejoy,2,False
3,Ned Flanders,2,False
4,Homer Simpson,2,False
5,Julius Hibbert,2,False


### Question 3

"Consider the HTML string < title > +++BREAKING NEWS +++ < title >. We would like to extract the first HTML tag (i.e.,
“< title >”). To do so we write the regular expression “<.+>”. Explain why this fails and correct the expression."

First I will demonstrate that the given regular expression does not work. Then I will explain the reasons that it doesn't work. Finally, I will correct the expression to extract the appropriate tag from the HTML string.


In [48]:
#first copy in the given expression that we are to test and assign a variable so it's easier to use later on
givenpattern = "<.+>"

#give the HTML string a variable as well that we will pass in to the search regex function
htmlstring = "<title>+++BREAKING NEWS+++<title>"

#use regex search function and .group to extract the string based on the pattern we were given to see the results
re.search(givenpattern, htmlstring).group()

'<title>+++BREAKING NEWS+++<title>'

**Explanation:** 
* "<" indicates that a string that matches will begin with a < 
* ">" indicates that the string will end with a >
* .+ will return any non-empty string within a single line and the + matches one or more occurrence of the pattern

In simpler words, the given expression does not work because of the second "< title >" in the HTML string. The expression recognizes the second "< title >" as another occurrence of the pattern. If we were to remove this second "< title >", the given expression would work, as shown in the code below. 

In [46]:
#copy the HTML string from above but remove the last <title> to illustrate point
htmlaltered = "<title>+++BREAKING NEWS+++"

#use the same expression as before but sub in the new altered html string variable
re.search(givenpattern, htmlaltered).group()

'<title>'

Now that I have verified that this is what is happening, I will correct the expression to extract the desired HTML tag.

In [231]:
#I will use one of the regular expressions from the DAV 5400 Week 8 Discussion Board to correct the expression
#this expression looks for a string starting with < then containing anything except > before closing with a >
#this expression will stop searching after one match so it will stop after the first "<title>"
pattern = "<[^>]+>"

#use the regex search function and pass the expression pattern above and the HTML string from above
#use the group function to extract the HTML tag
re.search(pattern, htmlstring).group()

'<title>'

### Question 4

"Consider the string “(5-3)^2=5^2-2x5x3+3^2” conforms to the binomial theorem. We would like to extract the
formula in the string. To do so we write the regular expression “[^0-9=+*()]+”. Explain why this fails and correct the
expression."

First I will test the given expression to see what happens and confirm that it does not work, then I will explain why it doesn't work. Lastly, I will correct the exression. 

In [65]:
#first copy the string from above and assign it to a variable to make it easier to use
binomial = "(5-3)^2=5^2-2*5*3+3^2"

#next copy the given expression and assign it to a variable for easy of use
givenexp = "[^0-9=+*()]+"

#use the regex search function and pass the given expression and the binomial string from above
#use the group function to extract the formula
re.search(givenexp, binomial).group()

'-'

**Explanation**
[^ will return anything EXCEPT for what follows the ^ symbol. So the expression as it stands will exclude any numbers (ex. 5 and 3), as well as parentheses, asterisks, + and = symbols. 
To correct this expression I simply need to add a - to the equation immediately following the 0-9 and move the ^ to anywhere in the expression after the new - symbol.

In [96]:
#create a new variable with the modified expression as described immediately above
newbinomexp = "[0-9-^=+*()]+"

#use the regex search function and pass the new expression and the binomial string from above
#use the group function to extract the formula
re.search(newbinomexp, binomial).group()

'(5-3)^2=5^2-2*5*3+3^2'

Now I will test another formula that adheres to the binomial theorem to make sure the new expression will successfully extract the new formula. 

In [97]:
#creates a new formula that adheres to the binomial theorem and assign it to the variable newbinomial
newbinomial = "(5-2)^3=5^3-3*5*2*2-3*5*2-2^3"

#test the new binomial using the regex search and group functions
re.search(newbinomexp, newbinomial).group()

'(5-2)^3=5^3-3*5*2*2-3*5*2-2^3'